In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
import pandas as pd
from transformers import pipeline

# Load your data
file_path = 'labeled_data.csv'
data = pd.read_csv(file_path)

In [2]:
# Check if GPU is available and set device accordingly
import torch
device = 0 if torch.cuda.is_available() else -1
device

0

In [3]:
# Initialize the pipelines for the models
abusive_pipe = pipeline("text-classification", model="Hate-speech-CNERG/english-abusive-MuRIL", device=device)
politeness_pipe = pipeline("text-classification", model="Genius1237/xlm-roberta-large-tydip", device=device)
threat_pipe = pipeline("text-classification", model="unitary/toxic-bert", device=device)

# Create functions to get abusiveness, politeness, and threat level scores
def get_abusiveness(text):
    result = abusive_pipe(text)[0]
    return result['score'] if result['label'] == 'LABEL_1' else 1 - result['score']

def get_politeness(text):
    result = politeness_pipe(text)[0]
    return result['score']

def get_threat_level(text):
    result = threat_pipe(text)[0]
    return result['score']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/950M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/521 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/811 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [4]:
# Apply the functions to the 'tweet' column and create new columns
data['abusiveness'] = data['tweet'].apply(get_abusiveness)


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [5]:
data.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,abusiveness
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,0.013545
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,0.992155
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,0.992184
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,0.989921
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,0.992193


In [6]:
data['politeness'] = data['tweet'].apply(get_politeness)


In [8]:
data.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,abusiveness,politeness
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,0.013545,0.803741
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,0.992155,0.886780
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,0.992184,0.951606
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,0.989921,0.884224
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,0.992193,0.869226


In [9]:
data.politeness.value_counts()

politeness
0.889769    3
0.954093    3
0.955706    3
0.954488    2
0.860434    2
           ..
0.877440    1
0.850590    1
0.921156    1
0.877485    1
0.984246    1
Name: count, Length: 24668, dtype: int64

In [10]:
data['threat_level'] = data['tweet'].apply(get_threat_level)

In [11]:
data.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,abusiveness,politeness,threat_level
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,0.013545,0.803741,0.736813
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,0.992155,0.886780,0.518833
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,0.992184,0.951606,0.998294
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,0.989921,0.884224,0.831770
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,0.992193,0.869226,0.969611


In [12]:
# Save the updated dataframe
data.to_csv('Finalized_data.csv', index=False)